<a href="https://colab.research.google.com/github/Hamza1122/Pose_Estimation_Yolo_Points/blob/main/testing_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import os
from sklearn import preprocessing
import cv2
import pandas as pd
import numpy as np


category = ['sami', 'sajid', 'amman', 'wajid', 'ahmed', 'shumail', 'steve', 'hannan']


In [112]:
#!unzip wajid.zip

In [113]:
data=r"/content/wajid/"
IMG_SIZE = 50

In [114]:
training_data = []
temp_array=[]
cate=["wajid"]
def create_training_data():
    for category in cate:
        
        path = os.path.join(data,category)
        class_num = cate.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                temp_array.append(img_array)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [115]:
training_data

[[array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 0],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 0],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 0],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 0],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0,

In [116]:

X_test=[]
y_test=[]


df_angle = pd.read_csv('wajid_angle.csv',names=['images','angles','distance'])
df_angle = df_angle.fillna(0)


df_test=pd.read_csv('wajid_pose.csv')

df_angle['path'] = df_test['Image']
df=pd.merge(df_test,df_angle, how='left', left_on='Image', right_on='path')   #merge both datasets and duplicates will be removed automatically
df=df.fillna(0)

In [117]:
df=df.drop(['Image','Class','images','path'],axis=1)

for features, label in training_data:
  X_test.append(features)
  y_test.append(label)

y_test=np.array(y_test)
len(y_test)




134

In [119]:
# min_max_scaler = preprocessing.MinMaxScaler()   #normalizing the data
# df = min_max_scaler.fit_transform(df)

In [ ]:
from keras.utils import np_utils

X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1) 
X_test=X_test.astype('float')
X_test=X_test/ 255 

y_test = np_utils.to_categorical(y_test)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy


json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
y_new = loaded_model.predict([df,X_test])

In [ ]:
results = np.argmax(y_new,axis=1)
results




In [31]:
min_max_scaler = preprocessing.MinMaxScaler()   #normalizing the data

In [102]:
def feature_extraction(training_data):
  X_test=[]
  y_test=[]

  for features, label in training_data:
    X_test.append(features)
    y_test.append(label)

  y_test=np.array(y_test)

  return X_test,y_test



def pre_processing(pose_path,angles_path):
  df_angle = pd.read_csv(angles_path,names=['images','angles','distance'])
  df_angle = df_angle.fillna(0)
  df_test=pd.read_csv(pose_path) 
  df_angle['path'] = df_test['Image']
  df=pd.merge(df_test,df_angle, how='left', left_on='Image', right_on='path')   #merge both datasets and duplicates will be removed automatically  
  df=df.fillna(0)
  return df

In [103]:
training_data = []
temp_array=[]

def create_training_data(data,cate):
    for category in cate:
        
        path = os.path.join(data,category)
        class_num = cate.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                temp_array.append(img_array)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

    return training_data          

In [104]:

def convolutional_neural_network(dataset_path,pose_path,angles_path):
  data=r"/content/"+dataset_path+'/'
  cate =[dataset_path]

  training_data = create_training_data(data,cate)
  
  df =pre_processing(pose_path,angles_path)
  df=df.drop(['Image','Class','images','path'],axis=1)
  
  min_max_scaler = preprocessing.MinMaxScaler()   #normalizing the data
  df = min_max_scaler.fit_transform(df)
  print(df)  

  X_test,y_test =feature_extraction(training_data)



In [105]:
convolutional_neural_network('wajid','wajid_pose.csv','wajid_angle.csv')

ValueError: ignored

In [4]:
data

'/content/wajid'